In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import nbimporter
import seaborn as sns


In [13]:
from library import load_data
data = load_data()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [14]:
#lets import and do prepocessing
from library import chess_bot_preprocess
processed_data = chess_bot_preprocess(data, remove_cols=True)

In [15]:
piece_to_num = {
    'P': 1,  # white pawn
    'N': 3,  # white knight
    'B': 3,  # white bishop
    'R': 5,  # white rook
    'Q': 9,  # white queen
    'K': 100, # white king
    'p': -1,  # black pawn
    'n': -3,  # black knight
    'b': -3,  # black bishop
    'r': -5,  # black rook
    'q': -9,  # black queen
    'k': -100,# black king
    '.': 0    # empty square
}



In [16]:
import chess
import numpy as np

def board_to_matrix(board):
    """
    Convert a chess.Board object to a numeric 8x8 matrix.
    Positive numbers = white, negative = black.
    """
    matrix = np.zeros((8,8), dtype=int)
    
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        row = 7 - (square // 8)  # chess library counts from bottom-left as 0
        col = square % 8
        if piece:
            matrix[row, col] = piece_to_num[piece.symbol()]
        else:
            matrix[row, col] = 0
    return matrix

In [17]:
board = chess.Board()
matrix = board_to_matrix(board)
print(matrix)

[[  -5   -3   -3   -9 -100   -3   -3   -5]
 [  -1   -1   -1   -1   -1   -1   -1   -1]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [   1    1    1    1    1    1    1    1]
 [   5    3    3    9  100    3    3    5]]


In [20]:
import pandas as pd
import chess
import numpy as np

def create_move_level_dataset(df):
    """
    Converts a game-level DataFrame into a move-level DataFrame.
    
    Each row corresponds to one move:
    - game_id: unique identifier of the game
    - move_num: number of the move in the game (0-indexed)
    - board: 8x8 numeric board before the move
    - move: [from_square, to_square]
    - extra_features: like white_rating, black_rating, turns, etc.
    """
    
    all_rows = []  # this will store all rows before making DataFrame
    
    for idx, row in df.iterrows():
        game_id = idx  # use DataFrame index as game ID
        moves_str = row['moves']
        
        # split the game moves into a list of UCI moves
        board = chess.Board()
        moves_list = []
        for move_san in moves_str.split():
            try:
                move = board.parse_san(move_san)  # SAN -> Move object
                moves_list.append(move.uci())      # store as UCI string
                board.push(move)                   # play move
            except:
                continue  # skip illegal moves
                
        # replay game move by move to record board state and move
        board = chess.Board()  # reset board
        for move_num, move_uci in enumerate(moves_list):
            board_matrix = board_to_matrix(board)          # 8x8 numeric matrix
            move_obj = chess.Move.from_uci(move_uci)
            move_array = move_to_array(move_obj)          # [from_square, to_square]
            
            # combine all info into a dict
            move_row = {
                'game_id': game_id,
                'move_num': move_num,
                'board': board_matrix,
                'move': move_array
            }
            
            # add extra features from original DataFrame
            for col in df.columns:
                if col != 'moves':  # don't repeat moves string
                    move_row[col] = row[col]
            
            all_rows.append(move_row)  # store this move
            
            board.push(move_obj)  # play the move
            
    # finally, make a new DataFrame
    move_level_df = pd.DataFrame(all_rows)
    return move_level_df

def move_to_array(move):
    """
    Convert a chess.Move object to a simple array:
    [from_square, to_square], where each square is 0-63.
    
    Parameters
    ----------
    move : chess.Move
    
    Returns
    -------
    list : [from_square, to_square]
    """
    return [move.from_square, move.to_square]


In [22]:
move_df = create_move_level_dataset(processed_data)  # check = your preprocessed game-level DataFrame
display(move_df.head())

,game_id,move_num,board,move,white_rating,black_rating,turns,victory_status,increment_code,opening_name,winner_num,rated_num
0,0,0,"[[-5, -3, -3, -9, -100, -3, -3, -5], [-1, -1, ...","[11, 27]",1500,1191,13,outoftime,15+2,Slav Defense: Exchange Variation,1.0,0
1,0,1,"[[-5, -3, -3, -9, -100, -3, -3, -5], [-1, -1, ...","[51, 35]",1500,1191,13,outoftime,15+2,Slav Defense: Exchange Variation,1.0,0
2,0,2,"[[-5, -3, -3, -9, -100, -3, -3, -5], [-1, -1, ...","[10, 26]",1500,1191,13,outoftime,15+2,Slav Defense: Exchange Variation,1.0,0
3,0,3,"[[-5, -3, -3, -9, -100, -3, -3, -5], [-1, -1, ...","[50, 42]",1500,1191,13,outoftime,15+2,Slav Defense: Exchange Variation,1.0,0
4,0,4,"[[-5, -3, -3, -9, -100, -3, -3, -5], [-1, -1, ...","[26, 35]",1500,1191,13,outoftime,15+2,Slav Defense: Exchange Variation,1.0,0


In [34]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import chess

# --- 1️⃣ Helper functions ---
piece_to_num = {
    'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9, 'K': 100,
    'p': -1, 'n': -3, 'b': -3, 'r': -5, 'q': -9, 'k': -100,
    '.': 0
}

def board_to_matrix(board):
    """Convert chess.Board to 8x8 numeric matrix"""
    matrix = np.zeros((8,8), dtype=int)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        row = 7 - (square // 8)
        col = square % 8
        matrix[row, col] = piece_to_num[piece.symbol()] if piece else 0
    return matrix

def decode_move(move_id):
    """Convert int move_id back to from_square and to_square"""
    from_square = move_id // 64
    to_square = move_id % 64
    return chess.Move(from_square, to_square)

# --- 2️⃣ Prepare dataset ---
X_boards = []
y_moves = []

for idx, row in move_df.iterrows():
    board_matrix = np.array(row['board'])  # convert the stored board to a numpy array
    X_boards.append(board_matrix)
    
    move_id = row['move']  # move as int
    y_moves.append(move_id)

# Convert to tensors
X_tensor = torch.tensor(np.array(X_boards), dtype=torch.float32).unsqueeze(1)  # (N,1,8,8)
y_tensor = torch.tensor(y_moves, dtype=torch.long)

# Dataset and DataLoader
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# --- 3️⃣ Simple model ---
import torch.nn as nn

class SimpleChessNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(8*8, 128)
        self.fc2 = nn.Linear(128, 4096)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleChessNet()